In [1]:
import torch
import numpy as np
import torchvision
import os
import cv2
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def read_files(X, Y, path, ans):
  files = os.listdir(path)
  for name in files:
    img = cv2.imread(path + '/' + name, 0)
    if img.shape != 0:
      img = cv2.resize(img, (256, 256))
      vect = img.reshape(1, 256 ** 2)
      vect = vect / 255.
      X = vect if (X is None) else np.vstack((X, vect)) 
      Y = np.append(Y, ans)
  return X, Y

In [4]:
path = "/content/drive/MyDrive/lesson1_dataset"
X = None
Y = np.array([])
X, Y = read_files(X, Y, path + "/logloss_0", 0)
X, Y = read_files(X, Y, path + "/logloss_1", 1)

In [5]:
class BoxesDataset(Dataset):
    def __init__(self, X, Y):
        self.x = torch.from_numpy(X)
        self.y = torch.from_numpy(Y)

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [6]:
dataset = BoxesDataset(X, Y)
train_set, test_set = torch.utils.data.random_split(dataset, [round(0.65*len(dataset)), round(0.35*len(dataset))])

train_loader = DataLoader(dataset=train_set, batch_size=8, drop_last=True, shuffle=True)
test_loader = DataLoader(dataset=test_set, shuffle=True)

In [7]:
for x, y in train_loader:
    print(x.shape, y.shape)

torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])
torch.Size([8, 65536]) torch.Size([8])


In [8]:
model = nn.Sequential(
    nn.Linear(65536, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 2),
    nn.Softmax()
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [10]:
model.cuda()

Sequential(
  (0): Linear(in_features=65536, out_features=1024, bias=True)
  (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.3, inplace=False)
  (3): ReLU()
  (4): Linear(in_features=1024, out_features=1024, bias=True)
  (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Dropout(p=0.3, inplace=False)
  (7): ReLU()
  (8): Linear(in_features=1024, out_features=1024, bias=True)
  (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Dropout(p=0.3, inplace=False)
  (11): ReLU()
  (12): Linear(in_features=1024, out_features=2, bias=True)
  (13): Softmax(dim=None)
)

In [11]:
epochs = 15
model.train()
for i in range(epochs):
    for j, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x.float())
        loss = criterion(y_pred, y.long())
        print(f"Epoch {i}\t iter {j}\t loss {loss}")
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0	 iter 0	 loss 0.6265265345573425
Epoch 0	 iter 1	 loss 0.5381142497062683
Epoch 0	 iter 2	 loss 0.6290380954742432
Epoch 0	 iter 3	 loss 0.654078483581543
Epoch 0	 iter 4	 loss 0.45528626441955566
Epoch 1	 iter 0	 loss 0.5847827196121216
Epoch 1	 iter 1	 loss 0.5614520907402039
Epoch 1	 iter 2	 loss 0.4915042817592621
Epoch 1	 iter 3	 loss 0.5418227910995483
Epoch 1	 iter 4	 loss 0.335675984621048
Epoch 2	 iter 0	 loss 0.32817739248275757
Epoch 2	 iter 1	 loss 0.3171457052230835
Epoch 2	 iter 2	 loss 0.5966716408729553
Epoch 2	 iter 3	 loss 0.3192328214645386
Epoch 2	 iter 4	 loss 0.5845376253128052
Epoch 3	 iter 0	 loss 0.48346254229545593
Epoch 3	 iter 1	 loss 0.43999242782592773
Epoch 3	 iter 2	 loss 0.4538801908493042
Epoch 3	 iter 3	 loss 0.44541919231414795
Epoch 3	 iter 4	 loss 0.33750224113464355
Epoch 4	 iter 0	 loss 0.6928684115409851
Epoch 4	 iter 1	 loss 0.3874188959598541
Epoch 4	 iter 2	 loss 0.316852867603302
Epoch 4	 iter 3	 loss 0.4597145915031433
Epoch 4	 iter

In [12]:
model.eval()
preds = []
true = []

for (x,y) in test_loader:
    x = x.cuda()
    y = y.cuda()
    preds.append(int(torch.max(model(x.float()).data, 1)[1]))
    true.append(int(y))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [13]:
print(true)
print(preds)

accuracy_score(true, preds)

[1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]
[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]


0.9565217391304348